In [7]:
import uuid
import json
import numpy as np

In [ ]:
ADE_PATH = "ADE_blink"
ade_codes = np.load("ADE_codes.npy", allow_pickle= True).item()
letters = ['A', 'B', 'C', 'D', 'E']
with open("point_label_mappings.json", 'r') as f:
    point_mapping = json.load(f)
with open("depth_qa_pairs.json", 'r') as f:
    qa = json.load(f)
with open("../../Data/train_depth_20k.json", 'r') as f:
    depth20k_data = json.load(f)

In [8]:
## create 1000 (500*2) multitask data of direct labeling + CoT
cot_data = []
for f in list(point_mapping.keys()):
    path = f"{ADE_PATH}/{f}"
    question = "Multiple points are circled on the image, labeled by letters beside each circle. Which point is the closest to the camera?"
    answer = qa[f]["answer"]
    answer = f"({answer})"
    
    ## direct labeling data
    new_entry = {
            "id": str(uuid.uuid4()),
            "image": path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"
                },
            ]
            }
    
    cot_data.append(new_entry)
    question += "\nTo answer this question, let's think through it step by step, and we know the image is 336 x 336. First, what are the coordinates of points in the image? Second, what is the depth map for the image? Which point has a higher pixel value on the depth map? Remember, higher values indicate that the point is closer to the camera."
    points_dict = point_mapping[f]
    num_points = len(points_dict)
    answer = ""
    for i in range(num_points):
        letter = letters[i]
        x, y = points_dict[letter]['coordinate']
        answer+= f"Point {letter} is located at (x = {x} y = {y}),"
    answer = answer[:-1] + "."
    gt = qa[f]["answer"] 
    depth_tokens = ade_codes[f"{f[:-4]}_depth.png"]
    answer += f"The depth map for the image is {depth_tokens}. Since point {gt} has a higher pixel value on the depth map, the answer is that point {gt} is closer to the camera."
    assert answer.count("<DEPTH") == 102
    
    ## cot data
    new_entry = {
            "id": str(uuid.uuid4()),
            "image": path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"
                },
            ]
            }


    cot_data.append(new_entry)




In [11]:
## make a single training file having the data for all 10 epochs sequentially

import random
epochs = 10
total_length = 20000
random.seed(0)
percents = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2 , 0.1]
total_cot_size = 0
annealing_data =[]
for epoch in range(epochs):
    percent = percents[epoch]
    epoch_depth_data = random.sample(depth20k_data, int(total_length * percent) )
    cot_length = total_length - int(total_length * percent)
    repeat = cot_length//len(cot_data)
    epoch_cot_data = cot_data * repeat
    total_epoch_data = epoch_depth_data + epoch_cot_data
    annealing_data += total_epoch_data
    total_cot_size+= len(epoch_cot_data)
    
    
    
print(len(annealing_data), total_cot_size)
    
    

200000 90000


In [51]:
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
write_json(annealing_data, "../../Data/train_depth_annealing_data.json")